In [1]:
pip install -r requirements.txt


  Using cached tensorboardx-2.6.4-py3-none-any.whl.metadata (6.2 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached medpy-0.5.2.tar.gz (156 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached timm-1.0.16-py3-none-any.whl.metadata (57 kB)
  Using cached einops-0.8.1-py3-none-any.whl.metadata (13 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)
INFO: pip is looking at multiple versions of tensorboardx to determine which version is compatible with other requirements. This could take a while.
  Using cached huggingface_hub-0.33.2-py3-none-any.whl.metadata (14 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using c


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms.functional import to_tensor
import matplotlib.pyplot as plt
from tqdm import tqdm



In [2]:
from networks.vision_transformer import SwinUnet


c:\Users\tasni\miniconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\tasni\miniconda3\envs\tf\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [3]:
class Config:
    # Path to dataset folders
    image_dir = "./Dataset/2D_Sliced_Images"
    mask_dir = "./Dataset/2D_Sliced_Masks"
    
    # Training settings
    img_size = 224
    num_classes = 3
    base_lr = 0.01
    batch_size = 4
    max_epochs = 100
    n_gpu = 1
    num_workers = 4
    eval_interval = 5
    seed = 42

    # Paths for checkpoints and pretrained model
    snapshot_path = "./swin_output"
    pretrained_ckpt = "./pretrained_ckpt/swin_tiny_patch4_window7_224.pth"

# Initialize config
args = Config()

# Create output directory if it doesn't exist
os.makedirs(args.snapshot_path, exist_ok=True)

# Set random seed for reproducibility
torch.manual_seed(args.seed)


In [4]:
from PIL import Image

class HepaticDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.image_files = sorted(os.listdir(image_dir))

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        mask_path = os.path.join(self.mask_dir, self.image_files[idx])

        image = Image.open(img_path).convert("L")
        mask = Image.open(mask_path)

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return {
            'image': image.float(),
            'label': mask.long().squeeze()
        }


In [5]:
train_transform = transforms.Compose([
    transforms.Resize((args.img_size, args.img_size)),
    transforms.ToTensor(),
])

train_dataset = HepaticDataset(
    image_dir=os.path.join(args.root_path, '2D_Sliced_Images'),
    mask_dir=os.path.join(args.root_path, '2D_Sliced_Masks'),
    transform=train_transform
)

train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=args.num_workers)


AttributeError: 'Config' object has no attribute 'root_path'

In [8]:
class DiceLoss(nn.Module):
    def __init__(self, num_classes):
        super(DiceLoss, self).__init__()
        self.num_classes = num_classes

    def forward(self, input, target, smooth=1e-5, softmax=True):
        if softmax:
            input = torch.softmax(input, dim=1)

        target_onehot = torch.eye(self.num_classes)[target].permute(0, 3, 1, 2).to(input.device)

        dims = (0, 2, 3)
        intersection = torch.sum(input * target_onehot, dims)
        cardinality = torch.sum(input + target_onehot, dims)

        dice = (2. * intersection + smooth) / (cardinality + smooth)
        return 1. - dice.mean()


In [16]:
# Step 1: Create the model (DO NOT pass pretrained here)
model = SwinUnet(config=config,
                 img_size=args.img_size,
                 num_classes=args.num_classes,
                 zero_head=True)

# Step 2: Load pretrained weights
model.load_from(config)

# Step 3: Wrap in DataParallel if needed
if args.n_gpu > 1:
    model = nn.DataParallel(model)

# Step 4: Move to GPU
model = model.cuda()


NameError: name 'config' is not defined

In [17]:
pip install addict


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
import yaml
from addict import Dict

# Load YAML file
with open('configs/swin_tiny_patch4_window7_224_lite.yaml', 'r') as f:
    config_dict = yaml.safe_load(f)

# Convert to dot-accessible format
config = Dict(config_dict)


In [23]:
print(args.img_size)

224


In [28]:
def to_2tuple(x):
    return x if isinstance(x, tuple) else (x, x)

In [30]:
 tupImagesize=to_2tuple(args.img_size)

In [34]:
# Initialize the model
model = SwinUnet(config=config,
                 img_size=tupImagesize,
                 num_classes=args.num_classes,
                 zero_head=True)

# Load pretrained checkpoint
# model.load_from(config)

# Use DataParallel if multiple GPUs
if args.n_gpu > 1:
    model = nn.DataParallel(model)

# Move to GPU
model = model.cuda()


SwinTransformerSys expand initial----depths:[2, 2, 2, 2];depths_decoder:[1, 2, 2, 2];drop_path_rate:0.2;num_classes:3


IndexError: tuple index out of range